In [ ]:
# helpful munging scripts
# Callin Switzer
# 15 Jan 2016


In [2]:
# import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
 #%qtconsole

In [4]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [ ]:
aa = file_choose()

In [ ]:
print aa

In [ ]:
os.chdir('/Volumes/GMD/BeeSonicationLearningWithAvery/BeeFrequencyLearning/')

In [ ]:
fldrs = [f for f in os.listdir(".") if not f.startswith('.')]

In [ ]:
fldrs

In [ ]:
fldrs.sort(key=lambda x: os.path.getmtime(x)) # sort by time created

In [ ]:
# fldrs.reverse()
fldrs

In [ ]:
import csv

In [ ]:
with open("/Users/callinswitzer/Desktop/output.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(np.transpose(np.array([fldrs])))

In [ ]:
np.transpose(np.array([fldrs])).shape

In [ ]:
file_choose()

In [ ]:
# copy files and exclude movies
import shutil

In [ ]:
os.chdir('/Volumes/GMD/19 Dec Bee Sonication Backup/BeeSonicationLearningWithAvery/BeeFrequencyLearning/')

In [ ]:
sourceDir = os.getcwd()
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_NoImages'
IGNORE_PATTERNS = ('*.pgm')


In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
sourceDir = '/Volumes/GMD/BeeSonicationLearningWithAvery/HeavyLightFlowers/'
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_HeavyLight'
IGNORE_PATTERNS = ('*.pgm')


In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
# pick up here

In [ ]:
sourceDir = '/Volumes/GMD/BeeSonicationLearningWithAvery/TestAndTrainFakeAndRealFlowers/'
targDir = '/Users/callinswitzer/Desktop/SonicationLearningWAvery_TestTrainDataset_noVid'
IGNORE_PATTERNS = ('*.pgm', '*.mp4')

In [ ]:
shutil.copytree(sourceDir, targDir, ignore= shutil.ignore_patterns(IGNORE_PATTERNS))

In [ ]:
# done

In [26]:
os.chdir('/Users/callinswitzer/Desktop/SonicationLearningWAvery_NoImages/')

In [27]:
mypath = os.getcwd()

In [28]:
# get folder list
folders = [f for f in os.listdir(mypath) if "." not in f]

In [63]:
# for each folder, open it, and read the ampFreq.txt file
for ii in range(len(folders)):

    # read contents of each folder
    fcontents = [f for f in os.listdir(folders[ii]) if not f.startswith('.')]

    # get ampFreq file(s). there should be one per folder
    ampFreqFile = [x for x in fcontents if 'ampFreq.txt' in x]

    if len(ampFreqFile) > 1: 
        print("PROBLEM " + str(ii))


    # read ampFreq file
    tmpDF = pd.read_table(os.path.join(folders[ii], ampFreqFile[0]), header = None, sep = ',')

    # add folder name to data frame
    tmpDF[7] = folders[ii]

    # add frequency information
    folderInFolder = [x for x in fcontents if not 'ampFreq.txt' in x]

    if len(folderInFolder) > 1: 
        print("PROBLEM")

    # list files in inner folder
    accRecFiles = [x for x in os.listdir(os.path.join(folders[ii], folderInFolder[0])) if not x.startswith('.')]

    tmpDF[8] = accRecFiles

    # remove wingbeats
    tmpDF2 = tmpDF.ix[(tmpDF[0] > 220) & (tmpDF[0] < 450)]

    # renumber index
    tmpDF2.index = (np.arange(1, tmpDF2.shape[0] + 1))

    # write to .csv
    tmpDF2.to_csv('/Users/callinswitzer/Desktop/FreqTrainCSVs/' + folders[ii] + '.csv', 
                 header = False, index = True)
    
    print ii

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231


In [64]:
## combine csv's all into a single file
csvDir = '/Users/callinswitzer/Desktop/FreqTrainCSVs/'

csvFiles = [f for f in os.listdir(csvDir) if f.endswith('.csv')]

In [89]:
np_array_list = []
for file_ in csvFiles:
    df = pd.read_csv(os.path.join(csvDir, file_),index_col=None, header=None)
    np_array_list.append(df.as_matrix())

comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)

In [90]:
big_frame.columns = ['index', 'freq', 'amp', 'datetime', 'rewNum', 'rewTF', 'lowRewAmp', 'highrewAmp', 'BeeNumCol', 'accFile']

In [91]:
big_frame.head()

,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile
0,1,400,0.10449,2016_11_28__11_09_43_099,1,T,0,5,BeeBlue2_28Nov2016_Hive4_high,2016_11_28__11_09_43_099_395_450_test.txt
1,2,420,0.30211,2016_11_28__11_09_43_694,2,T,0,5,BeeBlue2_28Nov2016_Hive4_high,2016_11_28__11_09_43_694_395_450_test.txt
2,3,350,0.17133,2016_11_28__11_09_44_424,2,F,0,5,BeeBlue2_28Nov2016_Hive4_high,2016_11_28__11_09_44_424_395_450_test.txt
3,4,400,0.60713,2016_11_28__11_09_44_931,3,T,0,5,BeeBlue2_28Nov2016_Hive4_high,2016_11_28__11_09_44_931_395_450_test.txt
4,5,350,0.21482,2016_11_28__11_09_49_468,3,F,0,5,BeeBlue2_28Nov2016_Hive4_high,2016_11_28__11_09_49_468_395_450_test.txt


In [92]:
# save big data frame
big_frame.to_csv('/Users/callinswitzer/Desktop/freqLearn.csv', header = True, index = False)

In [93]:
# check to see if datetime and accFile agree

eqs = [big_frame['datetime'][ii][1:25] == big_frame['accFile'][ii][0:24] for ii in range(len(big_frame['datetime']))]

In [94]:
np.sum(np.invert(eqs)) # should be zero if all of them match

0

In [95]:
# find the one that doesn't match
big_frame.ix[np.invert(eqs)] # now they all match

,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile


In [99]:
# make a new column for bee color, hive, reward frequency, date, and treatment (initial, high, low)

str1 = big_frame['BeeNumCol'][1]

In [100]:
str1

'BeeBlue2_28Nov2016_Hive4_high'

In [109]:
str2 = str1.split("Bee")[1]

In [113]:
beeColNum = str2.split(r'_')[0]

In [115]:
beeNum = [int(s) for s in beeColNum.split() if s.isdigit()]

In [116]:
beeNum

[]

In [117]:
beeColNum.split()

['Blue2']

In [118]:
import re

In [122]:
BeeNum = re.findall('\d+', beeColNum)

In [123]:
BeeNum

['2']

In [125]:
# get just bee color
from string import digits

In [126]:
col = beeColNum.translate(None, digits)
col

'Blue'

In [127]:
# get datetime column
dt1 = big_frame['datetime'][0]

In [128]:
dt1

' 2016_11_28__11_09_43_099'

In [129]:
from datetime import datetime

In [130]:
# string parse time
datetime_object = datetime.strptime(dt1, ' %Y_%m_%d__%H_%M_%S_%f')

In [131]:
datetime_object

datetime.datetime(2016, 11, 28, 11, 9, 43, 99000)

In [132]:
# string format time
datetime.strftime(datetime_object, "%Y-%m-%d %H:%M:%S.%f")

'2016-11-28 11:09:43.099000'